In [73]:
import torch

In [74]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

In [75]:
import torchvision
from torchvision import transforms

In [76]:
train_data_path = 'D:/Pytorch/Chp-01 Classification/train/'

In [77]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
    ])

train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transforms)

In [78]:
test_data_path = 'D:/Pytorch/Chp-01 Classification/test/'
val_data_path = 'D:/Pytorch/Chp-01 Classification/val/'

test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transforms)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transforms)

In [79]:
batch_size = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)

In [80]:
class SimpleNet(nn.Module):
    
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

simplenet = SimpleNet()

In [81]:
import torch.optim as optim

In [82]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

In [105]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device='cpu'):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            target = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item()
        training_loss /= epoch + 1
        
        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            valid_loss += loss.data.item()
            
            _, predicted = torch.max(output, dim=1)  # Get the index of the max log-probability
            print(f"{predicted.shape}, {targets.shape}")
            correct = torch.eq(predicted, targets).view(-1)
            # correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], target).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= epoch + 1
        
        print(f"Epoch: {epoch}, Training Loss: {training_loss}, Validation Loss: {valid_loss}, Accuracy: {num_correct/num_examples}")
        

In [106]:
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, test_data_loader)

torch.Size([64]), torch.Size([64])
torch.Size([64]), torch.Size([64])
torch.Size([32]), torch.Size([32])
Epoch: 0, Training Loss: 2.5380012542009354, Validation Loss: 1.7908848524093628, Accuracy: 0.775
torch.Size([64]), torch.Size([64])
torch.Size([64]), torch.Size([64])
torch.Size([32]), torch.Size([32])
Epoch: 1, Training Loss: 1.1209815852344036, Validation Loss: 0.9320246130228043, Accuracy: 0.7625
torch.Size([64]), torch.Size([64])
torch.Size([64]), torch.Size([64])
torch.Size([32]), torch.Size([32])
Epoch: 2, Training Loss: 0.6569179954628149, Validation Loss: 0.63410817583402, Accuracy: 0.76875
torch.Size([64]), torch.Size([64])
torch.Size([64]), torch.Size([64])
torch.Size([32]), torch.Size([32])
Epoch: 3, Training Loss: 0.4549100175499916, Validation Loss: 0.48699426651000977, Accuracy: 0.7625
torch.Size([64]), torch.Size([64])
torch.Size([64]), torch.Size([64])
torch.Size([32]), torch.Size([32])
Epoch: 4, Training Loss: 0.3031602218747139, Validation Loss: 0.3989985942840576

In [108]:
from PIL import Image

In [119]:
labels = ['cat', 'fish']

img = Image.open('D:\\Pytorch\\Chp-01 Classification\\val\\fish\\10.18.05_Manistee_Lake_Coho_004_small.jpg')
img = transforms(img)
img = img.unsqueeze(0)

for batch in test_data_loader:
    inputs, targets = batch
    for photos in inputs:
        prediction = simplenet(photos)
        prediction = prediction.argmax()
        print(labels[prediction])

prediction = simplenet(img)
prediction = prediction.argmax()
print(labels[prediction])

cat
cat
cat
cat
cat
fish
cat
cat
cat
cat
cat
cat
cat
cat
cat
cat
fish
cat
cat
cat
cat
cat
cat
cat
cat
cat
fish
cat
fish
cat
cat
fish
cat
cat
cat
fish
cat
cat
cat
fish
cat
cat
cat
cat
cat
cat
cat
cat
cat
fish
cat
cat
fish
cat
cat
cat
fish
cat
cat
cat
fish
fish
cat
fish
fish
cat
fish
cat
cat
cat
cat
cat
cat
fish
cat
fish
cat
cat
cat
cat
cat
cat
cat
cat
cat
fish
cat
fish
cat
fish
cat
fish
fish
fish
fish
fish
fish
fish
fish
cat
fish
fish
fish
cat
fish
cat
cat
fish
cat
cat
fish
fish
fish
fish
fish
cat
cat
cat
cat
cat
cat
fish
fish
cat
fish
fish
fish
fish
fish
fish
cat
fish
cat
cat
cat
fish
fish
cat
fish
fish
cat
fish
cat
fish
fish
fish
fish
fish
fish
fish
fish
cat
fish
fish
fish
fish
fish
fish
fish
fish
fish


In [130]:
torch.save(simplenet, 'D:/Pytorch/Chp-01 Classification/simplenet')

In [132]:
import os
os.getcwd()

'C:\\Users\\Sohai'